<a href="https://colab.research.google.com/github/cyclingdata/kaggle_fraud_modelling/blob/master/Kaggle_fraud_detection_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import pickle

In [0]:
gcs_data = "gs://data-fraud-detection/train_identity.csv"

In [0]:
# Authenticate.
from google.colab import auth
auth.authenticate_user()

In [0]:
!gsutil cp $gcs_data train.csv
dataset_identity=pd.read_csv('train.csv', low_memory=False)
!rm train.csv

Copying gs://data-fraud-detection/train_identity.csv...
- [1 files][ 25.3 MiB/ 25.3 MiB]                                                
Operation completed over 1 objects/25.3 MiB.                                     


In [0]:
dataset_identity["device_info_cut"]=dataset_identity.DeviceInfo.str[:2]

In [0]:
dataset_identity.columns

Index(['TransactionID', 'id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06',
       'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14',
       'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22',
       'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30',
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo', 'device_info_cut'],
      dtype='object')

In [0]:
gcs_data = "gs://data-fraud-detection/train_transaction.csv"
!gsutil cp $gcs_data train.csv
dataset_transaction=pd.read_csv('train.csv', low_memory=False)
!rm train.csv

Copying gs://data-fraud-detection/train_transaction.csv...
\ [1 files][651.7 MiB/651.7 MiB]   47.7 MiB/s                                   
Operation completed over 1 objects/651.7 MiB.                                    


In [0]:
dataset_transaction = pd.merge(dataset_transaction,dataset_identity,on="TransactionID",how='left')

In [0]:
dataset_transaction.shape

(590540, 435)

In [0]:
dataset_transaction.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,device_info_cut
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M,SA


In [0]:
dataset_transaction.columns.tolist()

['TransactionID',
 'isFraud',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'dist1',
 'dist2',
 'P_emaildomain',
 'R_emaildomain',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'C9',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D6',
 'D7',
 'D8',
 'D9',
 'D10',
 'D11',
 'D12',
 'D13',
 'D14',
 'D15',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'V29',
 'V30',
 'V31',
 'V32',
 'V33',
 'V34',
 'V35',
 'V36',
 'V37',
 'V38',
 'V39',
 'V40',
 'V41',
 'V42',
 'V43',
 'V44',
 'V45',
 'V46',
 'V47',
 'V48',
 'V49',
 'V50',
 'V51',
 'V52',
 'V53',
 'V54',
 'V55',
 'V56',
 'V57',
 'V58',
 'V59',
 'V60',
 'V61',
 'V62',
 'V63',
 'V64',
 'V

In [0]:
def coerce_to_unicode(x):
    if sys.version_info < (3, 0):
        if isinstance(x, str):
            return unicode(x,'utf-8')
        else:
            return unicode(x)
    else:
        return str(x)

categorical_features = ['card1', 'card2', 'card3','card4','card5','card6','addr1','addr2','P_emaildomain','R_emaildomain','ProductCD','M1','M2','M3','M4','M5','M6','M7','M2','M3','DeviceType','DeviceInfo',"device_info_cut"
                        ]
numerical_features = ['TransactionAmt','dist1','dist2','C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','D1','D2','D3','D4','D5','D6','D7','D8','D9'
                     ]


In [0]:
for val in range(12,39):
  categorical_features.append("id_"+str(val))
for val in range(1,340):
  numerical_features.append("V"+str(val))

In [0]:
dataset_transaction.ProductCD.value_counts()

W    439670
C     68519
R     37699
H     33024
S     11628
Name: ProductCD, dtype: int64

In [0]:
dataset_transaction.card4.value_counts()

visa                384767
mastercard          189217
american express      8328
discover              6651
Name: card4, dtype: int64

In [0]:
dataset_transaction.card1.value_counts()

7919     14932
9500     14162
15885    10361
17188    10344
15066     7945
12695     7091
12544     6773
6019      6771
2803      6141
7585      5334
10616     5172
12839     5129
3154      4614
2616      4410
18132     4209
9633      4158
15497     3977
16132     3929
2884      3873
16075     3748
11207     3693
10112     3560
7508      3490
10057     3166
12501     3152
7826      3006
16659     2988
12577     2891
5812      2818
7664      2792
         ...  
15229        1
7772         1
12456        1
16550        1
5218         1
3958         1
1911         1
7342         1
11944        1
8572         1
4369         1
15463        1
10621        1
11365        1
2937         1
9084         1
3169         1
3859         1
13182        1
17776        1
3449         1
5498         1
7547         1
4706         1
2657         1
12390        1
10341        1
10006        1
15743        1
17105        1
Name: card1, Length: 13553, dtype: int64

In [0]:
y = dataset_transaction.isFraud
X= dataset_transaction.drop("isFraud",axis=1)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train_values, y_test_values = train_test_split(X,y, test_size=0.2, random_state=42)

In [0]:
to_impute = {}
import math
for feature in numerical_features:
  value_to_impute = X_train[feature].median()
  if math.isnan(value_to_impute):
    numerical_features.remove(feature)
  else:
    X_train[feature].fillna(value_to_impute,inplace=True)
    X_test[feature].fillna(value_to_impute,inplace=True)
    to_impute[feature] = value_to_impute
print(to_impute)

numerical_features = to_impute.keys()

for feature in categorical_features:
  X_train[feature] = X_train[feature].astype("str")

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


{'TransactionAmt': 68.95, 'dist1': 8.0, 'dist2': 37.0, 'C1': 1.0, 'C2': 1.0, 'C3': 0.0, 'C4': 0.0, 'C5': 0.0, 'C6': 1.0, 'C7': 0.0, 'C8': 0.0, 'C9': 1.0, 'C10': 0.0, 'C11': 1.0, 'C12': 0.0, 'C13': 3.0, 'C14': 1.0, 'D1': 3.0, 'D2': 97.0, 'D3': 8.0, 'D4': 26.0, 'D5': 10.0, 'D6': 0.0, 'D7': 0.0, 'D8': 37.83333206176758, 'D9': 0.6666659712791443, 'V1': 1.0, 'V2': 1.0, 'V3': 1.0, 'V4': 1.0, 'V5': 1.0, 'V6': 1.0, 'V7': 1.0, 'V8': 1.0, 'V9': 1.0, 'V10': 0.0, 'V11': 0.0, 'V12': 1.0, 'V13': 1.0, 'V14': 1.0, 'V15': 0.0, 'V16': 0.0, 'V17': 0.0, 'V18': 0.0, 'V19': 1.0, 'V20': 1.0, 'V21': 0.0, 'V22': 0.0, 'V23': 1.0, 'V24': 1.0, 'V25': 1.0, 'V26': 1.0, 'V27': 0.0, 'V28': 0.0, 'V29': 0.0, 'V30': 0.0, 'V31': 0.0, 'V32': 0.0, 'V33': 0.0, 'V34': 0.0, 'V35': 1.0, 'V36': 1.0, 'V37': 1.0, 'V38': 1.0, 'V39': 0.0, 'V40': 0.0, 'V41': 1.0, 'V42': 0.0, 'V43': 0.0, 'V44': 1.0, 'V45': 1.0, 'V46': 1.0, 'V47': 1.0, 'V48': 0.0, 'V49': 0.0, 'V50': 0.0, 'V51': 0.0, 'V52': 0.0, 'V53': 1.0, 'V54': 1.0, 'V55': 1.0, 'V56

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [0]:
numerical_features

dict_keys(['TransactionAmt', 'dist1', 'dist2', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29', 'V30', 'V31', 'V32', 'V33', 'V34', 'V35', 'V36', 'V37', 'V38', 'V39', 'V40', 'V41', 'V42', 'V43', 'V44', 'V45', 'V46', 'V47', 'V48', 'V49', 'V50', 'V51', 'V52', 'V53', 'V54', 'V55', 'V56', 'V57', 'V58', 'V59', 'V60', 'V61', 'V62', 'V63', 'V64', 'V65', 'V66', 'V67', 'V68', 'V69', 'V70', 'V71', 'V72', 'V73', 'V74', 'V75', 'V76', 'V77', 'V78', 'V79', 'V80', 'V81', 'V82', 'V83', 'V84', 'V85', 'V86', 'V87', 'V88', 'V89', 'V90', 'V91', 'V92', 'V93', 'V94', 'V95', 'V96', 'V97', 'V98', 'V99', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106', 'V107', 'V108', 'V109', 'V110', 'V111', 'V112', 'V113', 'V114', 'V115'

In [0]:
to_shift=pd.DataFrame()
to_scale=pd.DataFrame()
for feature_name in numerical_features:
    shift = X_train[feature_name].mean()
    scale = X_train[feature_name].std()
    if scale == 0.:
        del X_train[feature_name]
        del X_test[feature_name]
        print ('Feature %s was dropped because it has no variance' % feature_name)
    else:
        print ('Rescaled %s with shift %s and scale %s' % (feature_name,shift,scale))
        #we save the rescale values
        to_shift.loc[0,feature_name]=shift
        to_scale.loc[0,feature_name]=scale
        X_train[feature_name] = (X_train[feature_name] - shift).astype(np.float64) / scale
        X_test[feature_name] = (X_test[feature_name] - shift).astype(np.float64) / scale

Rescaled TransactionAmt with shift 135.05436718755217 and scale 236.0305206915733
Rescaled dist1 with shift 52.81508449893318 and scale 243.34901814770367


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


Rescaled dist2 with shift 49.33853337623192 and scale 140.83425077005052
Rescaled C1 with shift 14.083614149761235 and scale 133.88236444873743
Rescaled C2 with shift 15.240157313645138 and scale 154.8306056988122
Rescaled C3 with shift 0.005664307244217157 and scale 0.1572955934375053
Rescaled C4 with shift 4.086044129102178 and scale 68.92936547292364
Rescaled C5 with shift 5.565499373454804 and scale 25.7476625206923
Rescaled C6 with shift 9.063971111186373 and scale 71.5775221321174
Rescaled C7 with shift 2.8448602126866938 and scale 61.881107237348076
Rescaled C8 with shift 5.141656365360517 and scale 95.646652277429
Rescaled C9 with shift 4.478737680766756 and scale 16.656861658158043
Rescaled C10 with shift 5.231779388356419 and scale 95.76408358546469
Rescaled C11 with shift 10.23209901107461 and scale 94.4995350994901
Rescaled C12 with shift 4.070079926846615 and scale 86.89807493950838
Rescaled C13 with shift 32.509470145968095 and scale 129.33198649508583
Rescaled C14 with s

In [0]:
min_cat = 400
valid_values = {}
for cat in categorical_features:
    counts=X_train[cat].value_counts()
    rare_values = counts[counts <= min_cat].index.tolist()
    print(rare_values)
    X_train[cat].replace(rare_values, "RARE_VALUE",inplace=True)
    counts=X_train[cat].value_counts()
    
    valid_values[cat] = counts.index.tolist()
    
for index,items in valid_values.items():
    print(index,", possible values :")
    print(items)
categorical_features = valid_values

['3605', '18366', '8868', '15757', '13844', '7069', '15111', '5365', '13749', '12615', '15813', '14182', '16883', '10086', '16085', '17217', '5017', '16485', '10186', '1164', '6816', '2801', '7680', '11465', '16873', '16630', '2538', '4151', '13964', '8691', '12002', '10776', '12970', '10895', '4153', '1284', '6832', '3867', '3166', '6697', '7468', '5167', '3056', '1333', '11653', '7322', '9885', '5751', '14165', '7669', '12582', '14128', '10854', '5409', '8613', '7482', '2646', '18350', '9043', '5254', '13526', '16876', '11204', '7800', '12725', '4571', '13143', '12116', '5629', '4702', '15592', '6877', '15359', '16346', '13413', '11837', '10313', '10908', '5492', '6021', '5583', '5901', '13623', '6394', '2704', '3484', '13234', '5170', '9561', '1546', '11319', '2249', '12570', '10075', '12184', '15678', '1359', '4425', '18375', '2261', '17020', '11690', '17635', '6292', '8140', '3579', '16923', '15333', '14502', '1618', '1556', '2408', '2263', '6194', '16746', '6542', '5286', '7900',

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


['134.0', '205.0', '167.0', '202.0', '271.0', '432.0', '571.0', '251.0', '297.0', '472.0', '496.0', '556.0', '414.0', '549.0', '411.0', '584.0', '464.0', '474.0', '272.0', '258.0', '550.0', '561.0', '599.0', '585.0', '372.0', '226.0', '559.0', '449.0', '381.0', '593.0', '350.0', '345.0', '460.0', '333.0', '325.0', '498.0', '462.0', '304.0', '324.0', '507.0', '564.0', '113.0', '320.0', '102.0', '184.0', '492.0', '454.0', '239.0', '440.0', '155.0', '191.0', '568.0', '565.0', '253.0', '110.0', '422.0', '572.0', '536.0', '152.0', '341.0', '210.0', '371.0', '586.0', '105.0', '231.0', '400.0', '311.0', '355.0', '141.0', '477.0', '308.0', '198.0', '112.0', '527.0', '373.0', '224.0', '551.0', '566.0', '501.0', '348.0', '106.0', '537.0', '277.0', '533.0', '265.0', '534.0', '560.0', '160.0', '491.0', '295.0', '374.0', '433.0', '475.0', '547.0', '504.0', '424.0', '339.0', '208.0', '214.0', '119.0', '133.0', '146.0', '523.0', '405.0', '315.0', '168.0', '197.0', '468.0', '222.0', '596.0', '286.0', 

In [0]:
X_train_new = X_train[numerical_features]
X_test_new = X_test[numerical_features]
for feature in categorical_features.keys():
    for category in categorical_features[feature]:
        new_feature = feature + "_" + category
        print(new_feature)
        X_train_new[new_feature]=np.where(X_train[feature]==category,1,0)
        X_test_new[new_feature]=np.where(X_test[feature]==category,1,0)

card1_RARE_VALUE
card1_7919

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/dist-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  



card1_9500
card1_17188
card1_15885
card1_15066
card1_12695
card1_12544
card1_6019
card1_2803
card1_7585
card1_10616
card1_12839
card1_3154
card1_2616
card1_18132
card1_9633
card1_15497
card1_16132
card1_2884
card1_16075
card1_11207
card1_10112
card1_7508
card1_10057
card1_12501
card1_7826
card1_16659
card1_12577
card1_5812
card1_7664
card1_7861
card1_4461
card1_6530
card1_8320
card1_5033
card1_15063
card1_7207
card1_10486
card1_10023
card1_7815
card1_9112
card1_16560
card1_9026
card1_13832
card1_16136
card1_7676
card1_4806
card1_12932
card1_11839
card1_1444
card1_4436
card1_2772
card1_17399
card1_2455
card1_1675
card1_14290
card1_11157
card1_17055
card1_3821
card1_1893
card1_17131
card1_18018
card1_1974
card1_1724
card1_9803
card1_13249
card1_2392
card1_8528
card1_15775
card1_17400
card1_15651
card1_11201
card1_14858
card1_3682
card1_18249
card1_2256
card1_12686
card1_15484
card1_1342
card1_12469
card1_9992
card1_1939
card1_6550
card1_9175
card1_9300
card1_4663
card1_16255
card1_8695


In [0]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_features=30,n_estimators=100,min_samples_split=100,min_samples_leaf =50,n_jobs=-1)
#from sklearn.tree import DecisionTreeClassifier
#rf= DecisionTreeClassifier(min_samples_split=500,min_samples_leaf=200)

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
config = tf.ConfigProto(intra_op_parallelism_threads=0, 
                        inter_op_parallelism_threads=0, 
                        allow_soft_placement=True)

session = tf.Session(config=config)
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr
batch_size=2048
epochs=30
model=Sequential()
model.add(Dense(30, input_dim=X_train_new.shape[1],
                kernel_initializer='normal',activation='relu'))

model.summary()
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])
rlrop = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=2)
callbacks = [rlrop,
             ModelCheckpoint(filepath='best_model.h5', 
                             monitor='val_loss', 
                             save_best_only=True)]
opt=keras.optimizers.adam(lr=0.1)
lr_metric = get_lr_metric(opt)

import tensorflow as tf
print(tf.logging.set_verbosity(tf.logging.INFO))

history = model.fit(X_train_new, y_train_values, epochs=epochs, 
                    batch_size=batch_size, 
                    verbose=1, validation_split=0.2,callbacks=callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 30)                34050     
Total params: 34,050
Trainable params: 34,050
Non-trainable params: 0
_________________________________________________________________
None
Train on 377945 samples, validate on 94487 samples
Epoch 1/30
377945/377945 [==============================] - 32s 86us/step - loss: 0.1842 - acc: 0.9612 - val_loss: 0.1105 - val_acc: 0.9708
Epoch 2/30
377945/377945 [==============================] - 32s 83us/step - loss: 0.1042 - acc: 0.9720 - val_loss: 0.1029 - val_acc: 0.9719
Epoch 3/30
377945/377945 [==============================] - 31s 82us/step - loss: 0.0977 - acc: 0.9732 - val_loss: 0.0986 - val_acc: 0.9727
Epoch 4/30
377945/377945 [==============================] - 32s 85us/step - loss: 0.0937 - acc: 0.9740 - val_loss: 0.0961 - val_acc: 0.9733
Epoch 5/30
377945/377945 [========================

In [0]:
pred =  model.predict(X_test_new)

NameError: ignored

In [0]:
from sklearn.metrics import roc_curve, auc

In [0]:
fpr,tpr,threshold=roc_curve(y_test_values,pred)

In [0]:
plt.plot(fpr,tpr)

In [0]:
auc(fpr,tpr)

In [0]:
gcs_data = "gs://data-fraud-detection/test_transaction.csv"
!gsutil cp $gcs_data test.csv
test_transaction=pd.read_csv('test.csv', low_memory=False)
!rm test.csv

In [0]:
gcs_data = "gs://data-fraud-detection/test_identity.csv"
!gsutil cp $gcs_data test.csv
test_identity=pd.read_csv('test.csv', low_memory=False)
!rm train.csv

In [0]:
test_identity["device_info_cut"]=test_identity.DeviceInfo.str[:2]

In [0]:
test_transaction = pd.merge(test_transaction,test_identity,on="TransactionID",how='left')

In [0]:
for feature in to_impute.keys():
  test_transaction[feature].fillna(to_impute[feature],inplace=True)
for feature in categorical_features:
  test_transaction[feature] = test_transaction[feature].astype("str")

In [0]:
to_scale

In [0]:
test_new = test_transaction[numerical_features]
for feature in categorical_features.keys():
    for category in categorical_features[feature]:
        new_feature = feature + "_" + category
        test_new[new_feature]=np.where(test_transaction[feature]==category,1,0)


In [0]:
pred_test= rf.predict_proba(test_new)[:,1].tolist()

In [0]:
submission=pd.DataFrame({"TransactionID": test_transaction.TransactionID,"isFraud":pred_test})

In [0]:
submission.head()

In [0]:
submission.to_csv("submission.csv",index=False)

In [0]:
!gsutil cp submission.csv gs://data-fraud-detection/submission.csv